In [1]:
import uproot
import matplotlib.pyplot as plt
import numpy as np

In [2]:
file=uproot.open('~/Downloads/DYJetsToMuMu_M-50_TuneCP5_14TeV-madgraphMLM-pythia8.root')['Events']

In [3]:
file.keys()

[b'run',
 b'luminosityBlock',
 b'event',
 b'HTXS_Higgs_pt',
 b'HTXS_Higgs_y',
 b'HTXS_stage1_1_cat_pTjet25GeV',
 b'HTXS_stage1_1_cat_pTjet30GeV',
 b'HTXS_stage1_1_fine_cat_pTjet25GeV',
 b'HTXS_stage1_1_fine_cat_pTjet30GeV',
 b'HTXS_stage_0',
 b'HTXS_stage_1_pTjet25',
 b'HTXS_stage_1_pTjet30',
 b'HTXS_njets25',
 b'HTXS_njets30',
 b'btagWeight_CSVV2',
 b'btagWeight_DeepCSVB',
 b'CaloMET_phi',
 b'CaloMET_pt',
 b'CaloMET_sumEt',
 b'ChsMET_phi',
 b'ChsMET_pt',
 b'ChsMET_sumEt',
 b'nCorrT1METJet',
 b'CorrT1METJet_area',
 b'CorrT1METJet_eta',
 b'CorrT1METJet_muonSubtrFactor',
 b'CorrT1METJet_phi',
 b'CorrT1METJet_rawPt',
 b'nElectron',
 b'Electron_deltaEtaSC',
 b'Electron_dr03EcalRecHitSumEt',
 b'Electron_dr03HcalDepth1TowerSumEt',
 b'Electron_dr03TkSumPt',
 b'Electron_dr03TkSumPtHEEP',
 b'Electron_dxy',
 b'Electron_dxyErr',
 b'Electron_dz',
 b'Electron_dzErr',
 b'Electron_eInvMinusPInv',
 b'Electron_energyErr',
 b'Electron_eta',
 b'Electron_hoe',
 b'Electron_ip3d',
 b'Electron_jetPtRelv2',
 

In [4]:
file.values()

[<TBranch b'run' at 0x7fcf07f055d0>,
 <TBranch b'luminosityBlock' at 0x7fcf07f05710>,
 <TBranch b'event' at 0x7fcf08472f10>,
 <TBranch b'HTXS_Higgs_pt' at 0x7fcf08472910>,
 <TBranch b'HTXS_Higgs_y' at 0x7fcf084c1550>,
 <TBranch b'HTXS_stage1_1_cat_pTjet25GeV' at 0x7fcf084c16d0>,
 <TBranch b'HTXS_stage1_1_cat_pTjet30GeV' at 0x7fcf084c1850>,
 <TBranch b'HTXS_stage1_1_fine_cat_pTjet25GeV' at 0x7fcf084c1a50>,
 <TBranch b'HTXS_stage1_1_fine_cat_pTjet30GeV' at 0x7fcf084c1c10>,
 <TBranch b'HTXS_stage_0' at 0x7fcf084c1690>,
 <TBranch b'HTXS_stage_1_pTjet25' at 0x7fcf084c1e90>,
 <TBranch b'HTXS_stage_1_pTjet30' at 0x7fcf08498050>,
 <TBranch b'HTXS_njets25' at 0x7fcf08498250>,
 <TBranch b'HTXS_njets30' at 0x7fcf084983d0>,
 <TBranch b'btagWeight_CSVV2' at 0x7fcf084984d0>,
 <TBranch b'btagWeight_DeepCSVB' at 0x7fcf084986d0>,
 <TBranch b'CaloMET_phi' at 0x7fcf08498910>,
 <TBranch b'CaloMET_pt' at 0x7fcf08498ad0>,
 <TBranch b'CaloMET_sumEt' at 0x7fcf08498c10>,
 <TBranch b'ChsMET_phi' at 0x7fcf08498d

In [5]:
file.show()

run                        (no streamer)              asdtype('>u4')
luminosityBlock            (no streamer)              asdtype('>u4')
event                      (no streamer)              asdtype('>u8')
HTXS_Higgs_pt              (no streamer)              asdtype('>f4')
HTXS_Higgs_y               (no streamer)              asdtype('>f4')
HTXS_stage1_1_cat_pTjet25GeV
                           (no streamer)              asdtype('>i4')
HTXS_stage1_1_cat_pTjet30GeV
                           (no streamer)              asdtype('>i4')
HTXS_stage1_1_fine_cat_pTjet25GeV
                           (no streamer)              asdtype('>i4')
HTXS_stage1_1_fine_cat_pTjet30GeV
                           (no streamer)              asdtype('>i4')
HTXS_stage_0               (no streamer)              asdtype('>i4')
HTXS_stage_1_pTjet25       (no streamer)              asdtype('>i4')
HTXS_stage_1_pTjet30       (no streamer)              asdtype('>i4')
HTXS_njets25               (no streamer)      

L1_SingleMuCosmics_OMTF    (no streamer)              asdtype('bool')
L1_SingleMuOpen            (no streamer)              asdtype('bool')
L1_SingleMuOpen_NotBptxOR  (no streamer)              asdtype('bool')
L1_SingleMuOpen_er1p1_NotBptxOR_3BX
                           (no streamer)              asdtype('bool')
L1_SingleMuOpen_er1p4_NotBptxOR_3BX
                           (no streamer)              asdtype('bool')
L1_SingleTau120er2p1       (no streamer)              asdtype('bool')
L1_SingleTau130er2p1       (no streamer)              asdtype('bool')
L1_TOTEM_1                 (no streamer)              asdtype('bool')
L1_TOTEM_2                 (no streamer)              asdtype('bool')
L1_TOTEM_3                 (no streamer)              asdtype('bool')
L1_TOTEM_4                 (no streamer)              asdtype('bool')
L1_TripleEG16er2p5         (no streamer)              asdtype('bool')
L1_TripleEG_16_12_8_er2p5  (no streamer)              asdtype('bool')
L1_TripleEG_16_15_

Now lets print some muon objects such as pt eta phi

In [6]:
muon_pt,muon_eta,muon_phi=file.arrays(["Muon_pt", "Muon_eta", "Muon_phi"],outputtype=tuple)

In [7]:
print(muon_pt, muon_eta, muon_phi)

[[28.913345] [] [52.494305 40.533558 4.394153] ... [54.169434 51.1125] [50.2459 38.491104] [108.90053 52.06664 3.3444448]] [[0.38287354] [] [0.57702637 0.4537964 -0.8736572] ... [-0.49017334 -1.6101074] [-0.125 -0.770874] [0.7303467 -0.050605774 0.90319824]] [[-0.614624] [] [2.459961 -0.72839355 1.8188477] ... [-0.65563965 -2.065918] [0.7928467 -2.1796875] [1.6374512 0.67126465 -0.63830566]]


Pt>20 abs(eta)<2.4

In [8]:
masking=np.logical_and(muon_pt>20,abs(muon_eta)<2.4)

In [9]:
masking

<JaggedArray [[True] [] [True True False] ... [True True] [True True] [True True False]] at 0x7fcf07924e90>

In [10]:
print(muon_pt[masking],muon_eta[masking],muon_phi[masking])

[[28.913345] [] [52.494305 40.533558] ... [54.169434 51.1125] [50.2459 38.491104] [108.90053 52.06664]] [[0.38287354] [] [0.57702637 0.4537964] ... [-0.49017334 -1.6101074] [-0.125 -0.770874] [0.7303467 -0.050605774]] [[-0.614624] [] [2.459961 -0.72839355] ... [-0.65563965 -2.065918] [0.7928467 -2.1796875] [1.6374512 0.67126465]]


GenPart_statusFlags 
gen status flags stored bitwise, bits are: 0 : isPrompt, 1 : isDecayedLeptonHadron, 2 : isTauDecayProduct, 3 : isPromptTauDecayProduct, 4 : isDirectTauDecayProduct, 5 : isDirectPromptTauDecayProduct, 6 : isDirectHadronDecayProduct, 7 : isHardProcess, 8 : fromHardProcess, 9 : isHardProcessTauDecayProduct, 10 : isDirectHardProcessTauDecayProduct, 11 : fromHardProcessBeforeFSR, 12 : isFirstCopy, 13 : isLastCopy, 14 : isLastCopyBeforeFSR,

In [12]:
genpart_status, genpart_statusflags, genpart_pdgid, genpart_pt, genpart_eta, genpart_phi=file.arrays(["GenPart_status", "GenPart_statusFlags", "GenPart_pdgId", "GenPart_pt", "GenPart_eta", "GenPart_phi"], outputtype=tuple)
masking=np.logical_and(abs(genpart_pdgid)==13, genpart_status==1)

step 3
which is same as before.

step 4 Gen matching
Gen matching information for muon is already stored in Muon_genPartIdx, which is matched on status=1 muon using particle flow algorithm. 
Lets print the Moun_genPartIdx. Note that -1 means not matched to generated particles.


In [15]:
muon_genpartidx=file.array("Muon_genPartIdx")
muon_genpartidx

<JaggedArray [[8] [] [14 16 -1] ... [13 12] [10 8] [10 9 -1]] at 0x7fcf0827a490>

We can also do a matching ourselves using deltaR method